In [1]:
import numpy as np
import itertools
import pandas as pd
from tqdm import tqdm
from time import sleep 

In [2]:
n = 10
cards = 5

In [24]:
df = pd.read_excel('data.xlsx', index_col=0, nrows = n)
real_name = {0 : 'BW', 1 : 'BE', 2 :'CRS', 3 : 'DM', 4 : 'ES',
             5 : 'IN', 6 : 'KRK', 7 : 'MH', 8 : 'SHR', 9 : 'WC'}
# df.index = [i for i in range(10)]
# df.rename({'BW': 0, 'BE': 1,'CRS': 2,'DM': 3,'ES': 4,
#            'IN': 5,'KRK': 6,'MH': 7,'SHR': 8,'WC': 9
#           }, axis=1, inplace=True)
df.fillna(0.5, inplace=True)
A = np.array(df)

In [4]:
# A = np.random.sample(n*n).reshape(n, n)
# for i in range(n):
#     for j in range(n):
#         if i < j:
#             A[i, j] = 1 - A[j, i]
#         if i == j:
#             A[i, i] = 0.5
# for i in range(10):
#     A[i] = df[i]
# pd.DataFrame(A)

In [5]:
my_hand = np.random.randint(0, n, cards+1)
my_hand

array([9, 6, 8, 9, 0, 7])

In [6]:
enemy_cards = np.random.randint(0, n, cards)
enemy_cards

array([9, 0, 2, 4, 0])

In [7]:
def compire(first, second):
    out = 0
    for comb in itertools.product('10', repeat=cards):
        count_ones = 0
        for i in range(cards):
            count_ones += int(comb[i]=='1')
        if count_ones > cards // 2:
            mul = 1
            for i, item in enumerate(comb):
                if item == '1':
                    mul *= A[first[i], second[i]]
                else:
                    mul *= A[second[i], first[i]]
            out += mul
    return out
# compire(first, second)

In [8]:
argmax = list()
max_prob = 0
for my_cards in itertools.permutations(my_hand, 5):
    probability = compire(my_cards, enemy_cards)
    if max_prob < probability:
        max_prob = probability
        argmax = my_cards
argmax = list(argmax)
print('My hand:', my_hand)
print('My cards:', argmax)
print('Enemy cards:', enemy_cards)
print('Probability:', max_prob)
result = pd.DataFrame({'My card' : [i for i in argmax],
                       'Enemy card' : [i for i in enemy_cards],
                       'Result': [A[argmax[i], enemy_cards[i]] for i in range(cards)]})
result

My hand: [9 6 8 9 0 7]
My cards: [7, 6, 9, 8, 0]
Enemy cards: [9 0 2 4 0]
Probability: 0.6143603083613127


,My card,Enemy card,Result
0,7,9,0.641026
1,6,0,0.538462
2,9,2,0.555556
3,8,4,0.500000
4,0,0,0.500000


In [9]:
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9
0,0.500000,0.568627,0.450000,0.648148,0.377593,0.474820,0.461538,0.500000,0.587302,0.619910
1,0.431373,0.500000,0.666667,0.333333,0.581081,0.386792,0.562500,0.450000,0.500000,0.520000
2,0.550000,0.333333,0.500000,0.666667,0.550000,0.500000,0.500000,0.500000,0.562500,0.444444
3,0.351852,0.666667,0.333333,0.500000,0.469565,0.553846,0.416667,0.483871,0.682927,0.408163
4,0.622407,0.418919,0.450000,0.530435,0.500000,0.569149,0.634615,0.553846,0.565217,0.469880
5,0.525180,0.613208,0.500000,0.446154,0.430851,0.500000,0.565217,0.413793,0.493506,0.470899
6,0.538462,0.437500,0.500000,0.583333,0.365385,0.434783,0.500000,0.750000,0.833333,0.611111
7,0.500000,0.550000,0.500000,0.516129,0.446154,0.586207,0.250000,0.500000,0.384615,0.641026
8,0.412698,0.500000,0.437500,0.317073,0.500000,0.506494,0.166667,0.615385,0.500000,0.540541
9,0.380090,0.480000,0.555556,0.591837,0.530120,0.529101,0.388889,0.358974,0.459459,0.500000


In [30]:
%%time
argmax = list()
max_prob = 0
all_obs = 0
dict_hands = {}
for _ in itertools.combinations_with_replacement([i for i in range(n)], cards):
    all_obs += 1
for first in tqdm(itertools.combinations_with_replacement([i for i in range(n)], cards), total=all_obs):
    sum_prob = 0
    for second in itertools.combinations_with_replacement([i for i in range(n)], cards):
        sum_prob += compire(first, second)
    dict_hands[tuple([real_name[x] for x in first])] = sum_prob / all_obs
    if sum_prob > max_prob:
        max_prob = sum_prob
        argmax = first
print(argmax)
print(max_prob / all_obs)

100%|██████████| 2002/2002 [04:22<00:00,  7.62it/s]

(4, 4, 4, 6, 6)
0.6230621281953846
CPU times: user 4min 21s, sys: 1.58 s, total: 4min 22s
Wall time: 4min 22s


In [42]:
sorted_by_value = sorted(dict_hands.items(), key=lambda x: x[1], reverse=True)
pd.DataFrame(sorted_by_value, columns=['Your hand', 'Probability to win']).set_index('Your hand').head(15)

,Probability to win
Your hand,
"(ES, ES, ES, KRK, KRK)",0.623062
"(BW, ES, ES, KRK, KRK)",0.620109
"(BW, CRS, ES, KRK, KRK)",0.618750
"(ES, ES, KRK, KRK, KRK)",0.617661
"(BW, BW, ES, KRK, KRK)",0.616616
"(IN, IN, KRK, KRK, KRK)",0.614886
"(BW, ES, KRK, KRK, KRK)",0.614842
"(BW, CRS, KRK, KRK, KRK)",0.613713
"(ES, IN, KRK, KRK, KRK)",0.613549
